# Control of a "spacecraft" platform with a reaction wheel

In [ ]:
# These are standard modules
import time
import numpy as np
from scipy import linalg
from scipy import signal
import sympy as sym
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Markdown

# This is a custom interface to the pybullet simulator
import ae353_platform

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

Given a choice of ground roll angle $\phi$, the equations of motion are

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - m_wgl\sin(q_1)\sin(\phi) \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + \left( \dfrac{J_2}{J_1} \right) m_wgl\sin(q_1)\sin(\phi)
\end{aligned}
$$

where

* $q_1$ is the platform angle
* $\dot{q}_1$ is the platform velocity
* $v_2$ is the wheel velocity
* $\tau$ is the torque applied *to* the platform *by* the wheel

and

$$
J_1 = J_p + m_wl^2
\qquad
\qquad
J_2 = J_w
$$

and parameter values are defined as follows:

In [ ]:
# distance from platform axis to wheel axis
l = 1.

# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

# mass of platform
mp = 12. * (1. - mw * l**2) / (3.**2 + 2.**2)

# moment of inertia of platform
Jp = (1. / 12.) * mp * (3.**2 + 2.**2)

# gravity
g = 9.81

# composite parameters
J1 = Jp + mw * l**2
J2 = Jw

Create an instance of the simulator (do `Kernel -> Restart` before evaluating this cell).

In [ ]:
simulator = ae353_platform.Simulator(
    display=True,
    roll=0.,
)

# Control wheel velocity only (no gravity)

Set ground roll angle of simulator.

In [ ]:
simulator.set_roll(0.)

The full equations of motion are:

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - m_wgl\sin(q_1)\sin(\phi) \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + \left( \dfrac{J_2}{J_1} \right) m_wgl\sin(q_1)\sin(\phi)
\end{aligned}
$$

The equations of motion in this particular case ($\phi=0$, ignore platform) are:

$$
\begin{aligned}
J_1 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau
\end{aligned}
$$

Write equations of motion as:

$$
\begin{bmatrix} \dot{v}_2 \end{bmatrix}
=
f \left( \begin{bmatrix} v_2 \end{bmatrix}, \begin{bmatrix} \tau \end{bmatrix} \right)
$$

Define a function to return a state-space model for a given choice of equilibrium point.

In [ ]:
def get_model(v2e, taue):
    # Define variables
    v2, tau = sym.symbols('v2, tau')
    
    # Define right-hand side of ODEs
    f = sym.Matrix([
        - ((J1 + J2) / (J1 * J2)) * tau,
    ])

    # Convert floating-point to rational numbers
    f = sym.nsimplify(f, rational=True)
    
    # Verify equilibrium point
    f_num = sym.lambdify([v2, tau], f)
    if not np.allclose(f_num(v2e, taue), 0.):
        raise Exception('equilibrium point is invalid')
    
    # Find A and B in symbolic form
    A_sym = f.jacobian([v2])
    B_sym = f.jacobian([tau])

    # Create lambda functions to allow numerical evaluation of A and B
    A_num = sym.lambdify([v2, tau], A_sym)
    B_num = sym.lambdify([v2, tau], B_sym)

    # Find A and B in numeric form (making sure the result is floating-point)
    A = A_num(v2e, taue).astype(float)
    B = B_num(v2e, taue).astype(float)

    # Return A and B
    return A, B

Choose equilibrium point and get state-space model.

In [ ]:
# Choose equilibrium point
(v2e, taue) = (2*np.pi, 0.)

# Get state-space model
A, B = get_model(v2e, taue)

# Show state-space model
print(f'A =\n{A}\n\nB =\n{B}')

Check if system is controllable.

In [ ]:
# Find controllability matrix
W = np.block([B])

# Check size of controllability matrix
print(f'shape of W is {W.shape}\n')

# Find determinant of controllability matrix
print(f'det(W) = {linalg.det(W)}\n')

# Find rank of controllability matrix
print(f'rank(W) = {np.linalg.matrix_rank(W)}')
print(f'number of states is {A.shape[0]}')

Do eigenvalue placement.

In [ ]:
# Choose desired eigenvalue locations
p = [-1.]

# Find gain matrix to put eigenvalues at desired locations
K = signal.place_poles(A, B, p).gain_matrix

# Show gain matrix
print(f'K =\n{K}\n')

# Show closed-loop eigenvalues
print(f'p = {linalg.eigvals(A - B @ K)}')

# Control platform angle only (no gravity)

Set ground roll angle of simulator.

In [ ]:
simulator.set_roll(0.)

The full equations of motion are:

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - m_wgl\sin(q_1)\sin(\phi) \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + \left( \dfrac{J_2}{J_1} \right) m_wgl\sin(q_1)\sin(\phi)
\end{aligned}
$$

The equations of motion in this particular case ($\phi=0$, ignore wheel) are:

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau
\end{aligned}
$$

Write equations of motion as:

$$
\begin{bmatrix} \dot{q}_1 \\ \dot{v}_1 \end{bmatrix}
=
f \left( \begin{bmatrix} q_1 \\ v_1 \end{bmatrix}, \begin{bmatrix} \tau \end{bmatrix} \right)
$$

Define a function to return a state-space model for a given choice of equilibrium point.

In [ ]:
def get_model(q1e, v1e, taue):
    # Define variables
    q1, v1, tau = sym.symbols('q1, v1, tau')
    
    # Define right-hand side of ODEs
    f = sym.Matrix([
        v1,
        (1 / J1) * tau,
    ])

    # Convert floating-point to rational numbers
    f = sym.nsimplify(f, rational=True)
    
    # Verify equilibrium point
    f_num = sym.lambdify([q1, v1, tau], f)
    if not np.allclose(f_num(q1e, v1e, taue), 0.):
        raise Exception('equilibrium point is invalid')
    
    # Find A and B in symbolic form
    A_sym = f.jacobian([q1, v1])
    B_sym = f.jacobian([tau])

    # Create lambda functions to allow numerical evaluation of A and B
    A_num = sym.lambdify([q1, v1, tau], A_sym)
    B_num = sym.lambdify([q1, v1, tau], B_sym)

    # Find A and B in numeric form (making sure the result is floating-point)
    A = A_num(q1e, v1e, taue).astype(float)
    B = B_num(q1e, v1e, taue).astype(float)

    # Return A and B
    return A, B

Choose equilibrium point and get state-space model.

In [ ]:
# Choose equilibrium point
(q1e, v1e, taue) = (np.pi / 6, 0., 0.)

# Get state-space model
A, B = get_model(q1e, v1e, taue)

# Show state-space model
print(f'A =\n{A}\n\nB =\n{B}')

Check if system is controllable.

In [ ]:
# Find controllability matrix
W = np.block([B, A @ B])

# Check size of controllability matrix
print(f'shape of W is {W.shape}\n')

# Find determinant of controllability matrix
print(f'det(W) = {linalg.det(W)}\n')

# Find rank of controllability matrix
print(f'rank(W) = {np.linalg.matrix_rank(W)}')
print(f'number of states is {A.shape[0]}')

Do eigenvalue placement.

In [ ]:
# Choose desired eigenvalue locations
p = [-1., -2.]

# Find gain matrix to put eigenvalues at desired locations
K = signal.place_poles(A, B, p).gain_matrix

# Show gain matrix
print(f'K =\n{K}\n')

# Show closed-loop eigenvalues
print(f'p = {linalg.eigvals(A - B @ K)}')

# Control platform angle and wheel velocity with gravity

Set ground roll angle of simulator.

In [ ]:
simulator.set_roll(np.pi / 2)

The full equations of motion are:

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - m_wgl\sin(q_1)\sin(\phi) \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + \left( \dfrac{J_2}{J_1} \right) m_wgl\sin(q_1)\sin(\phi)
\end{aligned}
$$

The equations of motion in this particular case ($\phi=\pi/2$) are:

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - m_wgl\sin(q_1) \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + \left( \dfrac{J_2}{J_1} \right) m_wgl\sin(q_1)
\end{aligned}
$$

Write equations of motion as:

$$
\begin{bmatrix} \dot{q}_1 \\ \dot{v}_1 \\ \dot{v}_2 \end{bmatrix}
=
f \left( \begin{bmatrix} q_1 \\ v_1 \\ v_2 \end{bmatrix}, \begin{bmatrix} \tau \end{bmatrix} \right)
$$

Define a function to return a state-space model for a given choice of equilibrium point.

In [ ]:
def get_model(q1e, v1e, v2e, taue):
    # Define variables
    q1, v1, v2, tau = sym.symbols('q1, v1, v2, tau')
    
    # Define right-hand side of ODEs
    f = sym.Matrix([
        v1,
        (1 / J1) * (tau - mw * g * l * sym.sin(q1)),
        (1 / J2) * (- ((J1 + J2) / J1) * tau + (J2 / J1) * mw * g * l * sym.sin(q1))
    ])

    # Convert floating-point to rational numbers
    f = sym.nsimplify(f, rational=True)
    
    # Verify equilibrium point
    f_num = sym.lambdify([q1, v1, v2, tau], f)
    if not np.allclose(f_num(q1e, v1e, v2e, taue), 0.):
        raise Exception('equilibrium point is invalid')
    
    # Find A and B in symbolic form
    A_sym = f.jacobian([q1, v1, v2])
    B_sym = f.jacobian([tau])

    # Create lambda functions to allow numerical evaluation of A and B
    A_num = sym.lambdify([q1, v1, v2, tau], A_sym)
    B_num = sym.lambdify([q1, v1, v2, tau], B_sym)

    # Find A and B in numeric form (making sure the result is floating-point)
    A = A_num(q1e, v1e, v2e, taue).astype(float)
    B = B_num(q1e, v1e, v2e, taue).astype(float)

    # Return A and B
    return A, B

Choose equilibrium point and get state-space model.

In [ ]:
# Choose equilibrium point
(q1e, v1e, v2e, taue) = (np.pi, 0., 0., 0.)

# Get state-space model
A, B = get_model(q1e, v1e, v2e, taue)

# Show state-space model
print(f'A =\n{A}\n\nB =\n{B}')

Check if system is controllable.

In [ ]:
# Find controllability matrix
W = np.block([B, A @ B, A @ A @ B])

# Check size of controllability matrix
print(f'shape of W is {W.shape}\n')

# Find determinant of controllability matrix
print(f'det(W) = {linalg.det(W)}\n')

# Find rank of controllability matrix
print(f'rank(W) = {np.linalg.matrix_rank(W)}')
print(f'number of states is {A.shape[0]}')

Do eigenvalue placement.

In [ ]:
# Choose desired eigenvalue locations
p = [-1., -2., -3.]

# Find gain matrix to put eigenvalues at desired locations
K = signal.place_poles(A, B, p).gain_matrix

# Show gain matrix
print(f'K =\n{K}\n')

# Show closed-loop eigenvalues
print(f'p = {linalg.eigvals(A - B @ K)}')

# Control platform angle and wheel velocity *without* gravity

Set ground roll angle of simulator.

In [ ]:
simulator.set_roll(0.)

The full equations of motion are:

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - m_wgl\sin(q_1)\sin(\phi) \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + \left( \dfrac{J_2}{J_1} \right) m_wgl\sin(q_1)\sin(\phi)
\end{aligned}
$$

The equations of motion in this particular case ($\phi=0$) are:

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau
\end{aligned}
$$

Write equations of motion as:

$$
\begin{bmatrix} \dot{q}_1 \\ \dot{v}_1 \\ \dot{v}_2 \end{bmatrix}
=
f \left( \begin{bmatrix} q_1 \\ v_1 \\ v_2 \end{bmatrix}, \begin{bmatrix} \tau \end{bmatrix} \right)
$$

Define a function to return a state-space model for a given choice of equilibrium point.

In [ ]:
def get_model(q1e, v1e, v2e, taue):
    # Define variables
    q1, v1, v2, tau = sym.symbols('q1, v1, v2, tau')
    
    # Define right-hand side of ODEs
    f = sym.Matrix([
        v1,
        (1 / J1) * (tau),
        (1 / J2) * (- ((J1 + J2) / J1) * tau)
    ])

    # Convert floating-point to rational numbers
    f = sym.nsimplify(f, rational=True)
    
    # Verify equilibrium point
    f_num = sym.lambdify([q1, v1, v2, tau], f)
    if not np.allclose(f_num(q1e, v1e, v2e, taue), 0.):
        raise Exception('equilibrium point is invalid')
    
    # Find A and B in symbolic form
    A_sym = f.jacobian([q1, v1, v2])
    B_sym = f.jacobian([tau])

    # Create lambda functions to allow numerical evaluation of A and B
    A_num = sym.lambdify([q1, v1, v2, tau], A_sym)
    B_num = sym.lambdify([q1, v1, v2, tau], B_sym)

    # Find A and B in numeric form (making sure the result is floating-point)
    A = A_num(q1e, v1e, v2e, taue).astype(float)
    B = B_num(q1e, v1e, v2e, taue).astype(float)

    # Return A and B
    return A, B

Choose equilibrium point and get state-space model.

In [ ]:
# Choose equilibrium point
(q1e, v1e, v2e, taue) = (np.pi, 0., 0., 0.)

# Get state-space model
A, B = get_model(q1e, v1e, v2e, taue)

# Show state-space model
print(f'A =\n{A}\n\nB =\n{B}')

Check if system is controllable.

In [ ]:
# Find controllability matrix
W = np.block([B, A @ B, A @ A @ B])

# Check size of controllability matrix
print(f'shape of W is {W.shape}\n')

# Find determinant of controllability matrix
print(f'det(W) = {linalg.det(W)}\n')

# Find rank of controllability matrix
print(f'rank(W) = {np.linalg.matrix_rank(W)}')
print(f'number of states is {A.shape[0]}')

Do eigenvalue placement.

In [ ]:
# Choose desired eigenvalue locations
p = [-1., -2., -3.]

# Find gain matrix to put eigenvalues at desired locations
K = signal.place_poles(A, B, p).gain_matrix

# Show gain matrix
print(f'K =\n{K}\n')

# Show closed-loop eigenvalues
print(f'p = {linalg.eigvals(A - B @ K)}')

#### What if $A$ was very, very slightly different?

In [ ]:
A2 = A + 1e-8 * np.random.standard_normal(size=A.shape)

print(f'A =\n{A}\n')
print(f'A2 =\n{A2}')

Check if system is controllable.

In [ ]:
# Find controllability matrix
W = np.block([B, A2 @ B, A2 @ A2 @ B])

# Check size of controllability matrix
print(f'shape of W is {W.shape}\n')

# Find determinant of controllability matrix
print(f'det(W) = {linalg.det(W)}\n')

# Find rank of controllability matrix
print(f'rank(W) = {np.linalg.matrix_rank(W)}')
print(f'number of states is {A.shape[0]}')

Do eigenvalue placement.

In [ ]:
# Choose desired eigenvalue locations
p = [-1., -2., -3.]

# Find gain matrix to put eigenvalues at desired locations
K = signal.place_poles(A2, B, p).gain_matrix

# Show gain matrix
print(f'K =\n{K}\n')

# Show closed-loop eigenvalues
print(f'p = {linalg.eigvals(A2 - B @ K)}')

Check singular values and condition number of $W$ instead of determinant and rank.

In [ ]:
print(f'singular values of W: {linalg.svdvals(W)}')

In [ ]:
print(f'condition number of W: {np.linalg.cond(W)}')

In [ ]:
s = linalg.svdvals(W)
ratio = max(s) / min(s)
print(f'ratio of largest singular value to smallest singlar value of W: {ratio}')

#### If a system is not controllable, does that mean we are completely out of luck?

In [ ]:
K = np.array([[1., 2., 3.]])

print(linalg.eigvals(A - B @ K))

In [ ]:
K = np.array([[11., 12., 13.]])

print(linalg.eigvals(A - B @ K))

# Control platform angle with *two* reaction wheels

In [ ]:
# Get state-space model
A = np.array([[0., 1.], [0., 0.]])
B = np.array([[0., 0.], [1., 10.]])

# Show state-space model
print(f'A =\n{A}\n\nB =\n{B}')

Check if system is controllable.

In [ ]:
# Find controllability matrix
W = np.block([B, A @ B])

# Check size of controllability matrix
print(f'shape of W is {W.shape}\n')

# # Find determinant of controllability matrix
# print(f'det(W) = {linalg.det(W)}\n')

# Find rank of controllability matrix
print(f'rank(W) = {np.linalg.matrix_rank(W)}')
print(f'number of states is {A.shape[0]}')

Choose linear combination of inputs.

In [ ]:
# First column of B
B_col1 = B[:, 0:1]

# Second column of B
B_col2 = B[:, 1:2]

# Multipliers
m1 = 0.1
m2 = 0.9

# Linear combination
B1 = m1 * B_col1 + m2 * B_col2

print(f'B1 =\n{B1}')

Do eigenvalue placement.

In [ ]:
# Choose desired eigenvalue locations
p = [-1., -2.]

# Find gain matrix to put eigenvalues at desired locations
K1 = signal.place_poles(A, B1, p).gain_matrix

# Find gain matrix for original system
K = np.block([[m1 * K1], [m2 * K1]])

# Show gain matrix
print(f'K =\n{K}\n')

# Show closed-loop eigenvalues
print(f'p = {linalg.eigvals(A - B @ K)}')

Could we have done eigenvalue placement with `place_poles` all at once?

In [ ]:
# Choose desired eigenvalue locations
p = [-1., -2.]

# Find gain matrix to put eigenvalues at desired locations
K = signal.place_poles(A, B, p).gain_matrix